In [43]:
%pip install -qU langchain langchain-openai python-dotenv rich

Note: you may need to restart the kernel to use updated packages.


In [44]:
from langchain.document_loaders import TextLoader

loader = TextLoader("document.txt")
document = loader.load()
print(document)

[
    Document(
        metadata={'source': 'document.txt'},
        page_content='# Document Indexing\n\nDocument indexing is a crucial technique in information retrieval that
organizes documents for efficient search and retrieval.\n\n\n## Why it is useful?\n\n- Enables fast and efficient 
document retrieval\n- Reduces search time complexity\n- Allows for complex queries and filtering\n- Supports 
scalable information retrieval systems\n\n\n## Common Indexing Techniques\n\n### Inverted Index\n- Maps terms to 
documents containing them\n- Stores frequency and position information\n- Optimized for full-text search\n\n\n### 
Vector Space Model\n- Represents documents as vectors\n- Enables similarity comparisons\n- Used with 
embedding-based search\n\n\n### Document-Term Matrix\n- Sparse matrix representation\n- Rows are documents, columns
are terms\n- Foundation for many IR algorithms\n'
    )
]

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(document)
print(documents)

[
    Document(metadata={'source': 'document.txt'}, page_content='# Document Indexing'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='Document indexing is a crucial technique in information retrieval that organizes documents 
for'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='documents for efficient search and retrieval.'),
    Document(metadata={'source': 'document.txt'}, page_content='## Why it is useful?'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='- Enables fast and efficient document retrieval\n- Reduces search time complexity'
    ),
    Document(
        metadata={'source': 'document.txt'},
        page_content='- Allows for complex queries and filtering\n- Supports scalable information retrieval 
systems'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='## Common Indexing Techniques'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='### Inverted Index\n- Maps terms to documents containing them'
    ),
    Document(
        metadata={'source': 'document.txt'},
        page_content='- Stores frequency and position information\n- Optimized for full-text search'
    ),
    Document(
        metadata={'source': 'document.txt'},
        page_content='### Vector Space Model\n- Represents documents as vectors\n- Enables similarity comparisons'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='- Used with embedding-based search'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='### Document-Term Matrix\n- Sparse matrix representation\n- Rows are documents, columns are 
terms'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='- Foundation for many IR algorithms')
]

In [46]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [47]:
from langchain.vectorstores.pgvector import PGVector
import os

COLLECTION_NAME = "my_collection"
DATABASE_URL = os.getenv("DATABASE_URL")

vector_store = PGVector.from_documents(
    documents=[],
    embedding=embeddings,
    connection_string=DATABASE_URL,
    collection_name=COLLECTION_NAME,
    create_extension=True,
    use_jsonb=True,
)

In [48]:
from langchain.indexes import SQLRecordManager, index
from rich import print

namespace = f"pgvector/{COLLECTION_NAME}"
record_manager = SQLRecordManager(namespace=namespace, db_url=DATABASE_URL)
record_manager.create_schema()

In [49]:
from rich import print

result = index(
    docs_source=documents,
    record_manager=record_manager,
    vector_store=vector_store,
    cleanup="incremental",
    source_id_key="source",
)
print(result)

{'num_added': 13, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [54]:
from langchain.schema import Document
from rich import print

documents[1].page_content = "updated"
del documents[0]
documents.append(Document(page_content="new content", metadata={"source": "important"}))
print(documents)

[
    Document(metadata={'source': 'document.txt'}, page_content='updated'),
    Document(metadata={'source': 'document.txt'}, page_content='documents for efficient search and retrieval.'),
    Document(metadata={'source': 'document.txt'}, page_content='## Why it is useful?'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='- Enables fast and efficient document retrieval\n- Reduces search time complexity'
    ),
    Document(
        metadata={'source': 'document.txt'},
        page_content='- Allows for complex queries and filtering\n- Supports scalable information retrieval 
systems'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='## Common Indexing Techniques'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='### Inverted Index\n- Maps terms to documents containing them'
    ),
    Document(
        metadata={'source': 'document.txt'},
        page_content='- Stores frequency and position information\n- Optimized for full-text search'
    ),
    Document(
        metadata={'source': 'document.txt'},
        page_content='### Vector Space Model\n- Represents documents as vectors\n- Enables similarity comparisons'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='- Used with embedding-based search'),
    Document(
        metadata={'source': 'document.txt'},
        page_content='### Document-Term Matrix\n- Sparse matrix representation\n- Rows are documents, columns are 
terms'
    ),
    Document(metadata={'source': 'document.txt'}, page_content='- Foundation for many IR algorithms'),
    Document(metadata={'source': 'important'}, page_content='new content')
]

In [55]:
from rich import print

result = index(
    docs_source=documents,
    record_manager=record_manager,
    vector_store=vector_store,
    cleanup="incremental",
    source_id_key="source",
)
print(result)

{'num_added': 2, 'num_updated': 0, 'num_skipped': 11, 'num_deleted': 2}

In [56]:
from rich import print

result = index(
    docs_source=[],
    record_manager=record_manager,
    vector_store=vector_store,
    cleanup="full",
    source_id_key="source",
)
print(result)

{'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 13}

In [57]:
from rich import print

result = index(
    docs_source=documents,
    record_manager=record_manager,
    vector_store=vector_store,
    cleanup="incremental",
    source_id_key="source",
)
print(result)

{'num_added': 13, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}